# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:


from models.mtcnn import MTCNN, fixed_image_standardization
from models.inception_resnet_v1 import  InceptionResnetV1
from models.utils import training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [2]:
data_dir = './data/test_mask'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

#### Determine if an nvidia GPU is available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [6]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
if os.path.isdir(data_dir + '_cropped') == False:
 
    loader = DataLoader(
             dataset,
             num_workers=workers,
             batch_size=batch_size,
             collate_fn=training.collate_pil
    )

    for i, (x, y) in enumerate(loader):
        mtcnn(x, save_path=y)
        print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
    # Remove mtcnn to reduce GPU memory usage
    del mtcnn

/media/user1/My4TBHD1/w251/W251_Project/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/media/user1/My4TBHD1/w251/W251_Project/models/mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  boxes = np.array(boxes)
/media/user1/My4TBHD1/w251/W251_Project/models/mtcnn.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is d

Batch 2 of 472

/media/user1/My4TBHD1/w251/W251_Project/models/mtcnn.py:444: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selected_boxes = np.array(selected_boxes)
/media/user1/My4TBHD1/w251/W251_Project/models/mtcnn.py:446: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selected_points = np.array(selected_points)


Batch 472 of 472

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [6]:

resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [7]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):int(0.9 * len(img_inds))]
test_inds = img_inds[int(0.9 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds),
    drop_last = True
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds),
    drop_last = True
)
test_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(test_inds),
    drop_last = True
)

#### Define loss and evaluation functions

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [9]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |    82/82   | loss:    3.7398 | fps:  507.7468 | acc:    0.0514   

Epoch 1/8
----------
Train |   658/658  | loss:    1.1308 | fps:  250.0223 | acc:    0.6799   
Valid |    82/82   | loss:    0.6498 | fps:  520.0208 | acc:    0.8205   

Epoch 2/8
----------
Train |   658/658  | loss:    0.5449 | fps:  244.3609 | acc:    0.8451   
Valid |    82/82   | loss:    0.5038 | fps:  501.1904 | acc:    0.8559   

Epoch 3/8
----------
Train |   658/658  | loss:    0.3975 | fps:  230.5265 | acc:    0.8839   
Valid |    82/82   | loss:    0.4782 | fps:  514.4096 | acc:    0.8590   

Epoch 4/8
----------
Train |   658/658  | loss:    0.2908 | fps:  244.1009 | acc:    0.9126   
Valid |    82/82   | loss:    0.3742 | fps:  534.6243 | acc:    0.8956   

Epoch 5/8
----------
Train |   658/658  | loss:    0.2269 | fps:  237.9029 | acc:    0.9309   
Valid |    82/82   | loss:    0.3715 | fps:  506.5142 | acc:    0.8895   

Epoch 6/8
----------
Train |   658/658  | loss:    0.0

In [10]:
print('\n Test set ')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, test_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)


 Test set 
----------
Valid |    82/82   | loss:    0.1757 | fps:  488.3806 | acc:    0.9554   


(tensor(0.1757), {'fps': tensor(488.3806), 'acc': tensor(0.9554)})

In [11]:
# Specify a path
PATH = "./data/model_with_mask.pt"

# Save
torch.save(resnet, PATH)